In [1]:
import aocd
from aocd.models import Puzzle
day = 23
year = 2021
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [24]:
from heapq import heappush, heappop

In [2]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(5,
 ['#############',
  '#...........#',
  '###C#C#B#D###',
  '  #D#A#B#A#',
  '  #########'])

In [15]:
test_data = """#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########"""

In [18]:
print(" "+"".join([str(i) for i in range(10)]))
print(test_data)

 0123456789
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########


In [260]:
start_test = (0, [],
         (
             (2,2), (8,2), 
             (2,1), (6,1),
             (4,1), (6,2),
             (4,2), (8,1)
         )
        )

In [261]:
start = (0, [],
         (
             (4,2), (8,2), 
             (6,1), (6,2),
             (2,1), (4,1),
             (2,2), (8,1)
         )
        )

In [267]:
def free_target(idx_target, positions):
    target_col = idx_target * 2
    for i, pos in enumerate(positions):
        x, y = pos
        idx = i//2 + 1
        if x == target_col and idx!= idx_target:
            return False
    return True

def check_finished(positions):
    for i, pos in enumerate(positions):
        x, y = pos
        idx = i//2 + 1
        if idx*2 != x:
            return False
    return True

def check_free(x1, x2, positions):
    xa, xb = sorted((x1, x2))
    for x in range(xa, xb+1):
        if x == x1:
            continue
        if (x, 0) in positions:
            return False
    return True

def replace(i, pos, positions):
#     print(i, pos, positions)
    new_positions = list(positions)
    new_positions[i] = pos
    return tuple(new_positions)
      

def gen_moves(positions):
#     print(positions)
    for i, pos in enumerate(positions):
        x, y = pos
        idx = i//2 + 1
        de = 10**(idx-1)
        if x == (idx*2) and y == 2:
            continue
            
        # try to move to target_col
        if y == 0:
            if not free_target(idx, positions):
                continue
            if check_free(x, idx*2, positions):
                if idx==99:
                    print("moving in {} {} | idx={} positions={}".format(x, y, idx, positions))
                dx = abs(x-idx*2)
                energy = dx*de
                if (idx*2, 2) not in positions:
                    yield (energy+2*de, replace(i, (idx*2, 2), positions))
                else:
                    yield (energy+1*de, replace(i, (idx*2, 1), positions))
                    
        # move out of the way
        else:
            if y == 2 and ((x, 1) in positions):
                continue
            energy = y*de
            for nsteps, xx in enumerate(range(x, 11)):
                if (xx, 0) in positions: 
                    break
                if xx not in (2, 4, 6, 8):
                    yield (energy + de*nsteps, replace(i, (xx, 0), positions))
            
            for nsteps, xx in enumerate(range(x, -1, -1)):
                if (xx, 0) in positions: 
                    break
                if  xx not in (2, 4, 6, 8):
                    yield (energy + de*nsteps, replace(i, (xx, 0), positions))
                    
            
    

def sim(start):
    minpos = 0
    maxpos = 10
    visited = set()
    target = (
         (2,1), (2,2), 
         (4,1), (4,2),
         (6,1), (6,2),
         (8,1), (8,2)
    )
    moves = []
    heappush(moves, start)
    nsteps = 0
    last = None
    while True:
        try:
            nxt = heappop(moves)
            last = nxt
            nsteps += 1
        except IndexError:
            print('no more moves', nsteps, energy)
            return last
        energy = nxt[0]
        path = nxt[1]
        positions = nxt[2]
        if len(set(positions)) < 8:
            print(positions)
        if positions in visited:
            continue
        visited.add(positions)

        if check_finished(positions):
            print('solution found')
            return energy, nsteps, positions, path#, moves
        for delta_energy, new_pos in gen_moves(positions):
#             if new_pos == target:
#                 return energy+delta_energy, len(visited)
#             heappush(moves, (energy+delta_energy, path + [(delta_energy, new_pos)], new_pos))
            heappush(moves, (energy+delta_energy, [], new_pos))
        if nsteps >= 1e6:
            return moves
#         break
    return visited

In [268]:
start

(0, [], ((4, 2), (8, 2), (6, 1), (6, 2), (2, 1), (4, 1), (2, 2), (8, 1)))

In [269]:
%%time
res = sim(start)

solution found
CPU times: user 17.9 s, sys: 0 ns, total: 17.9 s
Wall time: 17.9 s


In [270]:
res

(15299,
 676125,
 ((2, 1), (2, 2), (4, 1), (4, 2), (6, 1), (6, 2), (8, 1), (8, 2)),
 [])

In [266]:
puzzle.answer_a = res[0]

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [271]:
# Part B

In [434]:
from collections import defaultdict

In [466]:
def free_target(idx_target, positions):
    target_col = idx_target * 2
    for i, pos in enumerate(positions):
        x, y = pos
        idx = i//4 + 1
        if x == target_col and idx!= idx_target:
            return False
    return True

def check_finished(positions):
    for i, pos in enumerate(positions):
        x, y = pos
        idx = i//4 + 1
        if idx*2 != x:
            return False
    return True

def check_free(x1, x2, positions):
    xa, xb = sorted((x1, x2))
    for x in range(xa, xb+1):
        if x == x1:
            continue
        if (x, 0) in positions:
            return False
    return True

def replace(i, pos, positions):
#     print(i, pos, positions)
    new_positions = list(positions)
    new_positions[i] = pos
    return tuple(new_positions)
      

def gen_moves(positions):
#     print(positions)

    for i, pos in enumerate(positions):
        x, y = pos
        idx = i//4 + 1
        de = 10**(idx-1)
        
        if x == (idx*2) and y == 4:
            continue
            
        correct = 0    
        if x == (idx*2):
            si = (idx-1)*4
            same = positions[si:si+4]
            
            for yy in range(4, 0, -1):
                if (x, yy) in same:
                    correct += 1
                else:
                    break
            
            if correct >= 5-y:
                continue
                
            
        # try to move to target_col
        if y == 0:
            if not free_target(idx, positions):
                continue
            if check_free(x, idx*2, positions):
                if idx==99:
                    print("moving in {} {} | idx={} positions={}".format(x, y, idx, positions))
                dx = abs(x-idx*2)
                energy = dx*de
                
                for yy in range(1, 6):
                    if (idx*2, yy) in positions:
                        break
                yy -= 1
                yield (energy+yy*de, replace(i, (idx*2, yy), positions))
                                    
        # move out of the way
        else:
            for yy in range(y-1, 0, -1):
                if (x, yy) in positions:
                    break
            else:
                energy = y*de
                for nsteps, xx in enumerate(range(x, 11)):
                    if (xx, 0) in positions: 
                        break
                    if xx not in (2, 4, 6, 8):
                        yield (energy + de*nsteps, replace(i, (xx, 0), positions))

                for nsteps, xx in enumerate(range(x, -1, -1)):
                    if (xx, 0) in positions: 
                        break
                    if  xx not in (2, 4, 6, 8):
                        yield (energy + de*nsteps, replace(i, (xx, 0), positions))
                    
            
    
def sim(start):
    minpos = 0
    maxpos = 10
    visited = set()
    moves = []
    heappush(moves, start)
    nsteps = 0
    last = None
    while True:
        try:
            nxt = heappop(moves)
            last = nxt
            nsteps += 1
        except IndexError:
            print('no more moves', nsteps, energy)
            return last
        len_path = nxt[0]
        energy = nxt[1]
        path = nxt[2]
        if len_path > 50:
            print('max path length reached')
            return moves
        positions = nxt[3]
        
        if len(set(positions)) < 16:
            print(positions)
        if positions in visited:
            continue
        visited.add(positions)

        if check_finished(positions):
            print('solution found')
            return energy, nsteps, positions, path#, moves
        for delta_energy, new_pos in gen_moves(positions):
#             if new_pos == target:
#                 return energy+delta_energy, len(visited)
#             heappush(moves, (len(path)+1, energy+delta_energy, path + [(delta_energy, (set(positions)-set(new_pos), set(new_pos)-set(positions)))], new_pos))
            heappush(moves, (len_path+1, energy+delta_energy, None, new_pos))
#             heappush(moves, (energy+delta_energy, path+1, new_pos))
        if nsteps >= 1e7:
            print('max moves reached')
            return moves
#         break
    return visited

In [467]:
start_test_B = (0, 0,
 [],
 ((2, 4),
  (8, 4),
  (6, 3),
  (8, 2),
  (2, 1),
  (6, 1),
  (4, 3),
  (6, 2),
  (4, 1),
  (6, 4),
  (4, 2),
  (8, 3),
  (4, 4),
  (8, 1),
  (2, 2),
  (2, 3)))

In [468]:
start_B = (0, 0,
 [],
 ((4, 4),
  (8, 4),
  (6, 3),
  (8, 2),
  (6, 1),
  (6, 4),
  (4, 3),
  (6, 2),
  (2, 1),
  (4, 1),
  (4, 2),
  (8, 3),
  (2, 4),
  (8, 1),
  (2, 2),
  (2, 3)))

In [456]:
# test target: 44169

In [469]:
%%time
res = sim(start_B)

solution found
CPU times: user 27.3 s, sys: 44.5 ms, total: 27.3 s
Wall time: 27.3 s


In [470]:
start_B

(0,
 0,
 [],
 ((4, 4),
  (8, 4),
  (6, 3),
  (8, 2),
  (6, 1),
  (6, 4),
  (4, 3),
  (6, 2),
  (2, 1),
  (4, 1),
  (4, 2),
  (8, 3),
  (2, 4),
  (8, 1),
  (2, 2),
  (2, 3)))

In [473]:
res[0]

47193

In [472]:
len(res[-1]), res[-1]

TypeError: object of type 'NoneType' has no len()

In [474]:
puzzle.answer_b = res[0]

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 23! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [422]:
# init = start_test_B[3]
mv = list(gen_moves(init))

In [423]:
len(mv)

12

In [424]:
[(set(init)-set(m), set(m)-set(init), e) for e, m in mv ]

[({(6, 1)}, {(7, 0)}, 20),
 ({(6, 1)}, {(9, 0)}, 40),
 ({(6, 1)}, {(10, 0)}, 50),
 ({(4, 1)}, {(3, 0)}, 200),
 ({(4, 1)}, {(1, 0)}, 400),
 ({(4, 1)}, {(0, 0)}, 500),
 ({(8, 1)}, {(9, 0)}, 2000),
 ({(8, 1)}, {(10, 0)}, 3000),
 ({(8, 1)}, {(7, 0)}, 2000),
 ({(2, 2)}, {(3, 0)}, 3000),
 ({(2, 2)}, {(1, 0)}, 3000),
 ({(2, 2)}, {(0, 0)}, 4000)]